In [ ]:
# Importing libraries
import requests
import json
import polars as pl
from datetime import datetime
from utils import load_tokens, refresh_token_if_needed
import plotnine
from dataprep.eda import create_report

In [3]:
# Load token from persisted JSON
token = load_tokens()

In [4]:
# Initial authentication
access_token = refresh_token_if_needed(token['client_id'], ['client_secret'])

# Request activities data from Strava API
activities_url = 'https://www.strava.com/api/v3/athlete/activities'
headers = {'Authorization': f'Bearer {access_token}'}

# Loop through pages to get all activities
activities = []

page = 1
while True:
    response = requests.get(url     =   activities_url,
                            headers =   headers, 
                            params  =   {'per_page' : 100, 'page' : page}).json()
    
    if not response:
        break
    activities.extend(response)
    page += 1

activities = pl.DataFrame(activities)

No token refresh needed.


In [8]:
activities_curated = activities["id", "name", 
                   "distance", 
                   "moving_time", 
                   "total_elevation_gain", 
                   "sport_type","start_date", "gear_id", "start_latlng", "end_latlng",
                   "average_speed", "max_speed", "pr_count"]

In [11]:
activities_curated.filter(pl.col("sport_type") == "Ride")

id,name,distance,moving_time,total_elevation_gain,sport_type,start_date,gear_id,start_latlng,end_latlng,average_speed,max_speed,pr_count
i64,str,f64,i64,f64,str,str,str,list[f64],list[f64],f64,f64,i64
14461863801,"""Quick 10!""",10208.0,2072,11.6,"""Ride""","""2025-05-13T00:05:20Z""","""b15965201""","[43.185688, -79.238113]","[43.185237, -79.239943]",4.927,10.12,4
14415009088,"""20K along the canal 🏞️""",20550.0,4511,51.1,"""Ride""","""2025-05-08T10:56:40Z""","""b15965201""","[43.186056, -79.237056]","[43.185262, -79.239991]",4.556,9.96,1
14384045503,"""Morning Ride""",15950.8,4284,50.7,"""Ride""","""2025-05-05T10:38:13Z""","""b15965201""","[43.185237, -79.240003]","[43.185216, -79.239966]",3.723,9.02,3
14342156475,"""Port Dalhousie sunrise""",9518.0,2569,22.8,"""Ride""","""2025-05-01T09:58:17Z""","""b15965201""","[43.185268, -79.239972]","[43.185299, -79.239944]",3.705,9.18,1
14336914052,"""First ride in Nicasio +""",10092.8,2297,12.2,"""Ride""","""2025-04-30T20:18:07Z""","""b15965201""","[43.186227, -79.236506]","[43.185265, -79.239967]",4.394,9.24,0
…,…,…,…,…,…,…,…,…,…,…,…,…
1369817866,"""It's cold""",527.0,174,0.0,"""Ride""","""2018-01-22T16:23:43Z""",null,"[47.676095, -122.125355]","[47.675937, -122.125343]",3.029,6.3,0
425416333,"""Afternoon Ride""",3362.4,890,10.3,"""Ride""","""2015-10-18T21:29:41Z""",null,"[28.069222, -82.410976]","[28.069146, -82.376819]",3.778,6.7,0
376712576,"""Evening Ride""",4016.6,1550,0.0,"""Ride""","""2015-08-20T22:01:43Z""",null,"[28.055122, -82.413474]","[28.054901, -82.449535]",2.591,9.7,0


In [ ]:
activities_curated.filter(pl.col("start_date") <= '2025-01-01')

id,name,distance,moving_time,total_elevation_gain,sport_type,start_date,gear_id,start_latlng,end_latlng,average_speed,max_speed,pr_count
i64,str,f64,i64,f64,str,str,str,list[f64],list[f64],f64,f64,i64
12454252089,"""Nike Run Club: 5 Weeks To Go""",1697.5,900,0.0,"""Run""","""2024-09-19T21:56:36Z""","""g17360762""","[43.19, -79.24]","[43.2, -79.24]",1.886,3.584,0
12420582469,"""Nike Run Club: Sunday Evening …",1321.9,577,0.0,"""Run""","""2024-09-15T23:08:58Z""","""g17360762""","[43.19, -79.24]","[43.2, -79.24]",2.291,3.738,0
12403373307,"""Nike Run Club: 6 Weeks to Go -…",1618.7,766,0.0,"""Run""","""2024-09-14T00:04:20Z""","""g17360762""","[43.18, -79.24]","[43.2, -79.24]",2.113,5.376,0
12370126502,"""Nike Run Club: 6 Weeks to Go -…",1735.8,832,2.4,"""Run""","""2024-09-09T22:41:55Z""","""g17360762""","[43.19, -79.24]","[43.19, -79.24]",2.086,5.782,0
12363043634,"""Nike Run Club: Another Ten Mil…",16102.9,7765,26.0,"""Run""","""2024-09-08T23:16:32Z""","""g17360762""","[43.2, -79.21]","[43.2, -79.21]",2.074,7.25,3
…,…,…,…,…,…,…,…,…,…,…,…,…
12302958908,"""Nike Run Club: 15K Run""",15008.7,7145,6.0,"""Run""","""2024-09-02T00:24:43Z""","""g17360762""","[43.19, -79.24]","[43.19, -79.24]",2.101,7.416,0
12280805012,"""Nike Run Club: Power Pyramid""",3855.6,1619,5.0,"""Run""","""2024-08-30T12:05:38Z""","""g17360762""","[43.19, -79.24]","[43.19, -79.24]",2.381,5.872,0
12272383207,"""Nike Run Club: Breaking Throug…",4244.4,1769,3.0,"""Run""","""2024-08-29T11:51:57Z""","""g17360762""","[43.19, -79.24]","[43.19, -79.24]",2.399,6.786,0


In [13]:
activities_curated.head()

id,name,distance,moving_time,total_elevation_gain,sport_type,start_date,gear_id,start_latlng,end_latlng,average_speed,max_speed,pr_count
i64,str,f64,i64,f64,str,str,str,list[f64],list[f64],f64,f64,i64
14483079342,"""Evening Walk""",1327.4,1218,0.0,"""Walk""","""2025-05-14T22:07:48Z""",null,"[43.185118, -79.240331]","[43.18532, -79.240045]",1.09,2.5,0
14461863801,"""Quick 10!""",10208.0,2072,11.6,"""Ride""","""2025-05-13T00:05:20Z""","""b15965201""","[43.185688, -79.238113]","[43.185237, -79.239943]",4.927,10.12,4
14461614687,"""Afternoon Walk""",3182.6,2950,3.6,"""Walk""","""2025-05-08T21:14:11Z""",null,"[43.18215, -79.240976]","[43.185419, -79.238815]",1.079,7.52,0
14415009088,"""20K along the canal 🏞️""",20550.0,4511,51.1,"""Ride""","""2025-05-08T10:56:40Z""","""b15965201""","[43.186056, -79.237056]","[43.185262, -79.239991]",4.556,9.96,1
14384045503,"""Morning Ride""",15950.8,4284,50.7,"""Ride""","""2025-05-05T10:38:13Z""","""b15965201""","[43.185237, -79.240003]","[43.185216, -79.239966]",3.723,9.02,3


**Identify units for Strava data**
distance : meters
moving_time : seconds
total_elevation_gain : meters
average_speed : meters per second 
max_speed : meters per second 

In [17]:
activities_curated.head()

id,name,distance,moving_time,total_elevation_gain,sport_type,start_date,gear_id,start_latlng,end_latlng,average_speed,max_speed,pr_count
i64,str,f64,i64,f64,str,str,str,list[f64],list[f64],f64,f64,i64
14483079342,"""Evening Walk""",1327.4,1218,0.0,"""Walk""","""2025-05-14T22:07:48Z""",null,"[43.185118, -79.240331]","[43.18532, -79.240045]",1.09,2.5,0
14461863801,"""Quick 10!""",10208.0,2072,11.6,"""Ride""","""2025-05-13T00:05:20Z""","""b15965201""","[43.185688, -79.238113]","[43.185237, -79.239943]",4.927,10.12,4
14461614687,"""Afternoon Walk""",3182.6,2950,3.6,"""Walk""","""2025-05-08T21:14:11Z""",null,"[43.18215, -79.240976]","[43.185419, -79.238815]",1.079,7.52,0
14415009088,"""20K along the canal 🏞️""",20550.0,4511,51.1,"""Ride""","""2025-05-08T10:56:40Z""","""b15965201""","[43.186056, -79.237056]","[43.185262, -79.239991]",4.556,9.96,1
14384045503,"""Morning Ride""",15950.8,4284,50.7,"""Ride""","""2025-05-05T10:38:13Z""","""b15965201""","[43.185237, -79.240003]","[43.185216, -79.239966]",3.723,9.02,3


In [ ]:
# Filter bike rides only

activities_curated.filter(pl.col('sport_type') == 'Ride').gg

id,name,distance,moving_time,total_elevation_gain,sport_type,start_date,gear_id,start_latlng,end_latlng,average_speed,max_speed,pr_count
i64,str,f64,i64,f64,str,str,str,list[f64],list[f64],f64,f64,i64
14461863801,"""Quick 10!""",10208.0,2072,11.6,"""Ride""","""2025-05-13T00:05:20Z""","""b15965201""","[43.185688, -79.238113]","[43.185237, -79.239943]",4.927,10.12,4
14415009088,"""20K along the canal 🏞️""",20550.0,4511,51.1,"""Ride""","""2025-05-08T10:56:40Z""","""b15965201""","[43.186056, -79.237056]","[43.185262, -79.239991]",4.556,9.96,1
14384045503,"""Morning Ride""",15950.8,4284,50.7,"""Ride""","""2025-05-05T10:38:13Z""","""b15965201""","[43.185237, -79.240003]","[43.185216, -79.239966]",3.723,9.02,3
14342156475,"""Port Dalhousie sunrise""",9518.0,2569,22.8,"""Ride""","""2025-05-01T09:58:17Z""","""b15965201""","[43.185268, -79.239972]","[43.185299, -79.239944]",3.705,9.18,1
14336914052,"""First ride in Nicasio +""",10092.8,2297,12.2,"""Ride""","""2025-04-30T20:18:07Z""","""b15965201""","[43.186227, -79.236506]","[43.185265, -79.239967]",4.394,9.24,0
…,…,…,…,…,…,…,…,…,…,…,…,…
1369817866,"""It's cold""",527.0,174,0.0,"""Ride""","""2018-01-22T16:23:43Z""",null,"[47.676095, -122.125355]","[47.675937, -122.125343]",3.029,6.3,0
425416333,"""Afternoon Ride""",3362.4,890,10.3,"""Ride""","""2015-10-18T21:29:41Z""",null,"[28.069222, -82.410976]","[28.069146, -82.376819]",3.778,6.7,0
376712576,"""Evening Ride""",4016.6,1550,0.0,"""Ride""","""2015-08-20T22:01:43Z""",null,"[28.055122, -82.413474]","[28.054901, -82.449535]",2.591,9.7,0


In [ ]:
acti